[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1sYcpQ-QuhNlKRpEparQXUClDYHPt3_dQ#scrollTo=wyHyu9f4p0jK)

In [ ]:
!pip install -q -U  peft torch accelerate
!pip install bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
import torch
from peft import PeftModel, PeftConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,

)

notebook_login()

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
peft_model_id = "Elkhayyat17/qlora-med-llama2"
config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    use_auth_token=True,
    device_map="auto",
)




adapter_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

In [ ]:
def agent(inputs):


  messages = [{"role": "system", "content": '''Below is an instruction that describes a task, paired with an input
    that provides further context. Write a response that appropriately
    completes the request.'''},
  {"role": "user", "content": f" context:{inputs}"}]
  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Inference can be done using model.generate

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
  with torch.autocast("cuda", dtype=torch.float16):
    output = model.generate(
        input_ids=input_ids,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.01,
        return_dict_in_generate=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        repetition_penalty=1.2,
    )


  out = tokenizer.decode(output["sequences"][0], skip_special_tokens=True)
  start_index = out.find("[/INST]")

# Extract the portion starting from [/INST]
  desired_output = out[start_index+7:]


  return desired_output


In [ ]:
print(agent('''Create a nested loop to print every combination of numbers between 0-9?'''))


  To create a nested loop that prints every combination of numbers between 0 and 9, you can use two loops - one outer loop for iterating over the range of numbers (from 0 to 9), and another inner loop for iterating over each number in the outer loop's range. Here's an example code snippet:
```
for i in range(10): # Outer loop for iterating over the range of numbers from 0 to 9
    for j in range(i, i+2): # Inner loop for printing combinations of length 2 starting at index i


In [ ]:
print(agent('''python code to Print out the values in the following dictionary.",
            "my_dict = {\n  'name': 'John Doe',\n  'age': 32,\n  'city': 'New York'\n}"'''))


  Sure! To print out the values of this Python dictionary, you can use the `print()` function and pass it as an argument like so:
```
print(my_dict)
```


In [ ]:
print(agent("write an array of length 15 containing numbers divisible by 3 up to 45."))


  Sure! An array of lengths 15 containing numbers divisible by 3 would be:
[3,6,9,12,15].
